In [10]:
import os
import pandas as pd
import numpy as np
from IPython.display import display

data_path = os.path.join("data", "house_price.csv")
if not os.path.exists(data_path):
    raise FileNotFoundError(f"{data_path} not found. Put the CSV at that path and retry.")

# load (latin1 encoding used in your original cell)
df = pd.read_csv(data_path, encoding="latin1")
print("Loaded:", data_path, "shape:", df.shape)
print("\nTop rows:")
display(df.head())
print("\nColumn types:")
print(df.dtypes)
print("\nMissing values (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))
print("\nExample target columns present:", [c for c in ["totalPrice", "price", "square"] if c in df.columns])
# ...existing code...

Loaded: data\house_price.csv shape: (318851, 26)

Top rows:


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21476\1552017904.py:11: DtypeWarning: Columns (1,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, encoding="latin1")


,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,...,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,...,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,...,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,...,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,...,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0



Column types:
url                     object
id                      object
Lng                    float64
Lat                    float64
Cid                      int64
tradeTime               object
DOM                    float64
followers                int64
totalPrice             float64
price                    int64
square                 float64
livingRoom              object
drawingRoom             object
kitchen                  int64
bathRoom                object
floor                   object
buildingType           float64
constructionTime        object
renovationCondition      int64
buildingStructure        int64
ladderRatio            float64
elevator               float64
fiveYearsProperty      float64
subway                 float64
district                 int64
communityAverage       float64
dtype: object

Missing values (top 10):
DOM                  157977
buildingType           2021
communityAverage        463
subway                   32
fiveYearsProperty        32

In [11]:

# Stage 1: Basic preprocessing
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
# drop obvious irrelevant columns if they exist
drop_cols = [c for c in ['url','id','cid','link'] if c in df.columns]
if drop_cols:
    df = df.drop(columns=drop_cols)
# convert common price/area columns to numeric
def to_numeric_col(s):
    return pd.to_numeric(s.astype(str).str.replace(r'[^0-9.-]', '', regex=True), errors='coerce')

for col in ['totalprice','price','square','area','buildingarea']:
    if col in df.columns:
        df[col] = to_numeric_col(df[col])

# create binary target using median totalPrice if available (fall back to price)
target_col = 'totalprice' if 'totalprice' in df.columns else ('price' if 'price' in df.columns else None)
if target_col is None:
    raise RuntimeError("No price column found. Add totalPrice or price to proceed.")
median_price = df[target_col].median()
df['target'] = (df[target_col] >= median_price).astype(int)
print("Target defined using:", target_col, "median:", median_price)


Target defined using: totalprice median: 294.0


In [12]:

# Stage 2: Missing values, dtypes, derived features
# drop exact duplicates
df = df.drop_duplicates().reset_index(drop=True)

# impute numeric columns with median, categorical with mode
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.difference(['address','title']).tolist()

for c in num_cols:
    if df[c].isna().any():
        df[c].fillna(df[c].median(), inplace=True)
for c in cat_cols:
    if df[c].isna().any():
        df[c].fillna(df[c].mode().iloc[0] if not df[c].mode().empty else "missing", inplace=True)

# datetime parsing example (if present)
for date_col in ['publish_time','listing_date','built_year']:
    if date_col in df.columns:
        try:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        except Exception:
            pass

# derived features (defensive checks)
if 'square' in df.columns and target_col in df.columns:
    df['price_per_sqm'] = df[target_col] / df['square']
    df['log_price'] = (df[target_col].clip(lower=1)).apply(np.log)
# age of building (if build year or date available)
if 'built_year' in df.columns and pd.api.types.is_datetime64_any_dtype(df['built_year']):
    df['building_age'] = pd.Timestamp.now().year - df['built_year'].dt.year
elif 'built_year' in df.columns:
    df['building_age'] = pd.to_numeric(df['built_year'], errors='coerce').apply(lambda x: pd.Timestamp.now().year - x if pd.notna(x) else np.nan)
    df['building_age'].fillna(df['building_age'].median(), inplace=True)

print("After imputation shape:", df.shape)

After imputation shape: (318825, 26)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21476\2546533530.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(df[c].median(), inplace=True)


In [13]:

# Stage 3: Outliers and encoding
num_cols = df.select_dtypes(include=['number']).columns.tolist()
# exclude target columns from clipping
num_cols = [c for c in num_cols if c not in [target_col,'target']]
for c in num_cols:
    qlow, qhigh = df[c].quantile(0.01), df[c].quantile(0.99)
    if pd.notna(qlow) and pd.notna(qhigh) and qlow < qhigh:
        df[c] = df[c].clip(qlow, qhigh)

# select top categorical cols by cardinality (small ones for get_dummies)
cat_cols = df.select_dtypes(include=['object','category']).nunique().sort_values().index.tolist()
cat_to_encode = [c for c in cat_cols if df[c].nunique() <= 20][:6]  # keep up to 6 small-cardinal cols
if cat_to_encode:
    df = pd.get_dummies(df, columns=cat_to_encode, drop_first=True)

print("Columns after encoding:", len(df.columns))


Columns after encoding: 26


In [14]:

# Stage 4: Feature selection (top 10)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

# prepare feature matrix
features = df.select_dtypes(include=[np.number]).drop(columns=[target_col,'target'], errors='ignore').columns.tolist()

# corr ranking (abs correlation with continuous price if present)
corr_rank = pd.Series(0, index=features)
if target_col in df.columns:
    corr_vals = df[features + [target_col]].corr()[target_col].abs().drop(target_col)
    corr_rank = corr_vals.rank(ascending=False)

# RandomForest feature importance on binary target
X = df[features].fillna(0)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
imp = pd.Series(rf.feature_importances_, index=features)
imp_rank = imp.rank(ascending=False)

# combined rank average
combined_rank = (corr_rank.rank() + imp_rank.rank()) / 2
top10 = combined_rank.sort_values().head(10).index.tolist()
print("Top 10 features selected:", top10)

# create final dataframe with those features + target
final_cols = top10 + [target_col, 'target'] if target_col in df.columns else top10 + ['target']
final_df = df[final_cols].copy()

# save
out_path = os.path.join("data", "house_price_top10.csv")
final_df.to_csv(out_path, index=False)
print("Saved final dataset to:", out_path)


Top 10 features selected: ['log_price', 'price_per_sqm', 'price', 'square', 'communityaverage', 'dom', 'renovationcondition', 'ladderratio', 'followers', 'elevator']
Saved final dataset to: data\house_price_top10.csv
